In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime


In [6]:
combined_df

,origin_hdong_cd,dest_hdong_cd,date
0,1130563500,1130559500,20230901
1,1165058100,1165058000,20230901
2,1123053600,1111067000,20230901
3,1114052000,1111053000,20230901
4,1150060300,1147051000,20230901
...,...,...,...
32536786,1168070000,4161037000,20231015
32536787,1153056000,1153055000,20231015
32536788,4128161000,1171056600,20231015
32536789,1138063100,1144066000,20231015


##데이터 전처리

In [3]:
import pandas as pd
import glob

# 파일 경로들을 리스트로 관리
file_paths = glob.glob('rawdata/od_*.csv')  # 경로에 맞게 수정

# 모든 데이터를 읽어서 리스트에 저장
dataframes = []
total_files = len(file_paths)

print(f'총 {total_files}개의 파일이 있습니다.')

# 데이터를 순서대로 읽어들이고 진행 상황 출력
for i, file in enumerate(file_paths):
    print(f'{i+1}/{total_files} 파일 로드 중: {file}')
    
    df = pd.read_csv(file)
    dataframes.append(df)
    
    # Date 열의 결측값 확인
    missing_dates = df['date'].isnull().sum()
    print(f'Data{i+1} - Date 열 결측값: {missing_dates}')
    
    # 진행 상황 출력
    print(f'{i+1}/{total_files} 진행 완료.')

print("모든 파일 로드 및 결측값 확인 완료.")


총 46개의 파일이 있습니다.
1/46 파일 로드 중: rawdata\od_20230901_1.csv
Data1 - Date 열 결측값: 0
1/46 진행 완료.
2/46 파일 로드 중: rawdata\od_20230902_1.csv
Data2 - Date 열 결측값: 0
2/46 진행 완료.
3/46 파일 로드 중: rawdata\od_20230903_1.csv
Data3 - Date 열 결측값: 0
3/46 진행 완료.
4/46 파일 로드 중: rawdata\od_20230904_1.csv


MemoryError: Unable to allocate 248. MiB for an array with shape (9, 3606349) and data type int64

In [ ]:
NData.isnull().sum()


origin_hdong_cd    0
dest_hdong_cd      0
date               0
start_time         0
end_time           0
gender             0
age                0
modal              0
origin_purpose     0
dest_purpose       0
od_dist_avg        0
od_duration_avg    0
od_cnts            0
dtype: int64

In [ ]:
origin=[]
dest=[]
for i in range(1,46):
    if i>=40:
        filename='od_202310'+str(i-30)+'_1.csv'
    elif i>=31:
        filename='od_2023100'+str(i-30)+'_1.csv'
    elif i>=10:
        filename='od_202309'+str(i)+'_1.csv'
    else:
        filename='od_2023090'+str(i)+'_1.csv'
    Data=pd.read_csv('rawdata/'+str(filename))
    MData=Data[Data.origin_hdong_cd == 1.114054e+09]
    # 먼저 시간을 적절히 슬라이싱한 후에, datetime 형식으로 변환
    MData['start_time'] = pd.to_datetime(MData['start_time'], format='%H:%M')
    MData['end_time'] = pd.to_datetime(MData['end_time'], format='%H:%M')
    MData = MData.sort_values(by='start_time')
    MData['start_time'] = MData['start_time'].dt.hour
    MData['end_time'] = MData['end_time'].dt.hour
    MData=MData[MData.origin_purpose == 3]
    MData=MData[abs(MData.start_time-13)<=10]
    MData_sums = MData.select_dtypes(include=[int, float]).sum(axis=0)
    k=MData_sums['od_cnts']
    origin.append(k)
    TData=Data[Data.dest_hdong_cd == 1.114054e+09]
    TData['start_time'] = pd.to_datetime(TData['start_time'], format='%H:%M')
    TData['end_time'] = pd.to_datetime(TData['end_time'], format='%H:%M')
    TData = TData.sort_values(by='end_time')
    TData['start_time'] = TData['start_time'].dt.hour
    TData['end_time'] = TData['end_time'].dt.hour
    TData=TData[TData.dest_purpose == 3]
    TData=TData[abs(TData.end_time-13)<=10]
    TData_sums = TData.select_dtypes(include=[int, float]).sum(axis=0)
    k2=TData_sums['od_cnts']
    dest.append(k2)
print(origin)
print(dest)
import pandas as pd

# 평일 데이터
WOdata = pd.DataFrame()
WDdata = pd.DataFrame()

# 주말 데이터
EOdata = pd.DataFrame()
EDdata = pd.DataFrame()

# 추석 데이터
COdata = pd.DataFrame()
CDdata = pd.DataFrame()

# 회현동 코드
hoehyeon_code = 1.114054e+09

# 파일 수
total_files = 45

for i in range(1, total_files + 1):
    if i >= 40:
        filename = 'od_202310' + str(i-30) + '_1.csv'
    elif i >= 31:
        filename = 'od_2023100' + str(i-30) + '_1.csv'
    elif i >= 10:
        filename = 'od_202309' + str(i) + '_1.csv'
    else:
        filename = 'od_2023090' + str(i) + '_1.csv'
    
    # 파일 읽기
    Data = pd.read_csv('rawdata/' + filename)
    
    # 출발지와 도착지에서 회현동(남대문시장) 관련 데이터 필터링
    OData = Data[Data['origin_hdong_cd'] == hoehyeon_code]
    DData = Data[Data['dest_hdong_cd'] == hoehyeon_code]

    # 데이터 처리: 추석 기간(28일~33일), 주말, 평일 구분하여 데이터 추가
    if 28 <= i <= 33:  # 추석 데이터 처리
        COdata = pd.concat([COdata, OData], axis=0)
        CDdata = pd.concat([CDdata, DData], axis=0)
    elif i % 7 == 2 or i % 7 == 3 or i == 39:  # 주말 데이터 처리
        EOdata = pd.concat([EOdata, OData], axis=0)
        EDdata = pd.concat([EDdata, DData], axis=0)
    else:  # 평일 데이터 처리
        WOdata = pd.concat([WOdata, OData], axis=0)
        WDdata = pd.concat([WDdata, DData], axis=0)

    # 진행 상황 출력
    print(f'[{i}/{total_files}] 파일 처리 완료: {filename}')

# 각 데이터셋의 크기 및 일부 확인
print("\n최종 결과:")
print("평일 출발 데이터(WOdata):", WOdata.shape)
print("평일 도착 데이터(WDdata):", WDdata.shape)
print("주말 출발 데이터(EOdata):", EOdata.shape)
print("주말 도착 데이터(EDdata):", EDdata.shape)
print("추석 출발 데이터(COdata):", COdata.shape)
print("추석 도착 데이터(CDdata):", CDdata.shape)


C:\Users\ansck\AppData\Local\Temp\ipykernel_22640\636364054.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MData['start_time'] = pd.to_datetime(MData['start_time'], format='%H:%M')
C:\Users\ansck\AppData\Local\Temp\ipykernel_22640\636364054.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MData['end_time'] = pd.to_datetime(MData['end_time'], format='%H:%M')
C:\Users\ansck\AppData\Local\Temp\ipykernel_22640\636364054.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

[3962.0, 7898.0, 3602.0, 3349.0, 3527.0, 3364.0, 3506.0, 3950.0, 8586.0, 3660.0, 2863.0, 2970.0, 2957.0, 3437.0, 2988.0, 6426.0, 3386.0, 2744.0, 3483.0, 2590.0, 3924.0, 4097.0, 10140.0, 5143.0, 3319.0, 3355.0, 2682.0, 4401.0, 1301.0, 3681.0, 6892.0, 9546.0, 6584.0, 3267.0, 4215.0, 3963.0, 8277.0, 5959.0, 7298.0, 3651.0, 3902.0, 4518.0, 4155.0, 6027.0, 4199.0]
[3644.0, 8508.0, 3616.0, 3341.0, 3786.0, 3376.0, 3865.0, 3793.0, 9416.0, 4145.0, 2741.0, 3335.0, 2951.0, 3438.0, 2651.0, 7395.0, 4462.0, 3419.0, 3702.0, 2932.0, 4087.0, 4080.0, 11677.0, 5690.0, 3832.0, 4305.0, 2572.0, 5078.0, 1736.0, 3680.0, 7550.0, 10670.0, 7190.0, 2748.0, 4101.0, 3785.0, 9160.0, 6429.0, 8341.0, 3898.0, 4169.0, 4729.0, 4165.0, 6558.0, 5578.0]


C:\Users\ansck\AppData\Local\Temp\ipykernel_22640\636364054.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MData['start_time'] = pd.to_datetime(MData['start_time'], format='%H:%M')
C:\Users\ansck\AppData\Local\Temp\ipykernel_22640\636364054.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MData['end_time'] = pd.to_datetime(MData['end_time'], format='%H:%M')
C:\Users\ansck\AppData\Local\Temp\ipykernel_22640\636364054.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [ ]:
import pandas as pd

# 평일 데이터
WOdata = pd.DataFrame()
WDdata = pd.DataFrame()

# 주말 데이터
EOdata = pd.DataFrame()
EDdata = pd.DataFrame()

# 추석 데이터
COdata = pd.DataFrame()
CDdata = pd.DataFrame()

# 회현동 코드
hoehyeon_code = 1.114054e+09

# 파일 수
total_files = 45

for i in range(1, total_files + 1):
    if i >= 40:
        filename = 'od_202310' + str(i-30) + '_1.csv'
    elif i >= 31:
        filename = 'od_2023100' + str(i-30) + '_1.csv'
    elif i >= 10:
        filename = 'od_202309' + str(i) + '_1.csv'
    else:
        filename = 'od_2023090' + str(i) + '_1.csv'
    
    # 파일 읽기
    Data = pd.read_csv('rawdata/' + filename)
    
    # 출발지와 도착지에서 회현동(남대문시장) 관련 데이터 필터링
    OData = Data[Data['origin_hdong_cd'] == hoehyeon_code]
    DData = Data[Data['dest_hdong_cd'] == hoehyeon_code]

    # 데이터 처리: 추석 기간(28일~33일), 주말, 평일 구분하여 데이터 추가
    if 28 <= i <= 33:  # 추석 데이터 처리
        COdata = pd.concat([COdata, OData], axis=0)
        CDdata = pd.concat([CDdata, DData], axis=0)
    elif i % 7 == 2 or i % 7 == 3 or i == 39:  # 주말 데이터 처리
        EOdata = pd.concat([EOdata, OData], axis=0)
        EDdata = pd.concat([EDdata, DData], axis=0)
    else:  # 평일 데이터 처리
        WOdata = pd.concat([WOdata, OData], axis=0)
        WDdata = pd.concat([WDdata, DData], axis=0)

    # 진행 상황 출력
    print(f'[{i}/{total_files}] 파일 처리 완료: {filename}')

# 각 데이터셋의 크기 및 일부 확인
print("\n최종 결과:")
print("평일 출발 데이터(WOdata):", WOdata.shape)
print("평일 도착 데이터(WDdata):", WDdata.shape)
print("주말 출발 데이터(EOdata):", EOdata.shape)
print("주말 도착 데이터(EDdata):", EDdata.shape)
print("추석 출발 데이터(COdata):", COdata.shape)
print("추석 도착 데이터(CDdata):", CDdata.shape)


[1/45] 파일 처리 완료: od_20230901_1.csv
[2/45] 파일 처리 완료: od_20230902_1.csv
[3/45] 파일 처리 완료: od_20230903_1.csv
[4/45] 파일 처리 완료: od_20230904_1.csv
[5/45] 파일 처리 완료: od_20230905_1.csv
[6/45] 파일 처리 완료: od_20230906_1.csv
[7/45] 파일 처리 완료: od_20230907_1.csv
[8/45] 파일 처리 완료: od_20230908_1.csv
[9/45] 파일 처리 완료: od_20230909_1.csv
[10/45] 파일 처리 완료: od_20230910_1.csv
[11/45] 파일 처리 완료: od_20230911_1.csv
[12/45] 파일 처리 완료: od_20230912_1.csv
[13/45] 파일 처리 완료: od_20230913_1.csv
[14/45] 파일 처리 완료: od_20230914_1.csv
[15/45] 파일 처리 완료: od_20230915_1.csv
[16/45] 파일 처리 완료: od_20230916_1.csv
[17/45] 파일 처리 완료: od_20230917_1.csv
[18/45] 파일 처리 완료: od_20230918_1.csv
[19/45] 파일 처리 완료: od_20230919_1.csv
[20/45] 파일 처리 완료: od_20230920_1.csv
[21/45] 파일 처리 완료: od_20230921_1.csv
[22/45] 파일 처리 완료: od_20230922_1.csv
[23/45] 파일 처리 완료: od_20230923_1.csv
[24/45] 파일 처리 완료: od_20230924_1.csv
[25/45] 파일 처리 완료: od_20230925_1.csv
[26/45] 파일 처리 완료: od_20230926_1.csv
[27/45] 파일 처리 완료: od_20230927_1.csv
[28/45] 파일 처리 완료: od_20230928_1.csv
[

In [ ]:
print(EDdata.isnull().sum()['date'])
print(WDdata.isnull().sum()['date'])
print(CDdata.isnull().sum()['date'])
print(EOdata.isnull().sum()['date'])
print(WOdata.isnull().sum()['date'])
print(COdata.isnull().sum()['date'])


0
0
0
0
0
0


In [ ]:
WDdata.isnull().sum()

origin_hdong_cd    0
dest_hdong_cd      0
date               0
start_time         0
end_time           0
gender             0
age                0
modal              0
origin_purpose     6
dest_purpose       0
od_dist_avg        0
od_duration_avg    0
od_cnts            0
dtype: int64

In [ ]:
WDdata

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
347,1117051000,1114054000,20230901,08:00,08:00,1,3,0.0,0.0,0,13990,19,13
953,1117053000,1114054000,20230901,12:00,13:00,0,4,0.0,1.0,1,19979,41,8
1594,1114052000,1114054000,20230901,12:00,12:00,0,2,1.0,1.0,1,17380,9,12
1974,1114052000,1114054000,20230901,12:00,12:00,1,2,4.0,1.0,1,1278,6,34
3244,1114055000,1114054000,20230901,12:00,12:00,0,2,0.0,4.0,4,7137,11,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3690054,1114052000,1114054000,20231013,17:00,17:00,1,5,0.0,1.0,3,13285,19,5
3690788,1117051000,1114054000,20231013,09:00,09:00,1,5,0.0,1.0,1,2483,5,5
3693728,1114052000,1114054000,20231013,11:00,11:00,1,3,3.0,1.0,1,1895,7,5
3695251,1117053000,1114054000,20231013,17:00,18:00,1,3,0.0,1.0,1,19429,40,5


In [ ]:
WDdata.isnull().sum()

origin_hdong_cd    0
dest_hdong_cd      0
date               0
start_time         0
end_time           0
gender             0
age                0
modal              0
origin_purpose     6
dest_purpose       0
od_dist_avg        0
od_duration_avg    0
od_cnts            0
dtype: int64

In [ ]:
CO = COdata
CD = CDdata
EO = EOdata
ED = EDdata
WO = WOdata
WD = WDdata

In [ ]:
# 각 데이터프레임의 'date' 열을 int64로 변환하는 작업
CO['date'] = CO['date'].astype('str')
CD['date'] = CD['date'].astype('str')
EO['date'] = EO['date'].astype('str')
ED['date'] = ED['date'].astype('str')
WO['date'] = WO['date'].astype('str')
WD['date'] = WD['date'].astype('str')


In [ ]:
import pandas as pd

# 함수 정의: 'date'와 'start_time', 'end_time'을 결합하여 'start_datetime'과 'end_datetime' 생성
def process_dataframe(df):
    # 'date'를 datetime 형식으로 변환
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='coerce')

    # 'start_time'과 'end_time'을 시각 형식으로 변환
    df['start_time'] = pd.to_datetime(df['start_time'], format='%H:%M', errors='coerce').dt.time
    df['end_time'] = pd.to_datetime(df['end_time'], format='%H:%M', errors='coerce').dt.time

    # 'date'와 'start_time', 'end_time'을 결합하여 시계열 생성
    df['start_datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['start_time'].astype(str), errors='coerce')
    df['end_datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['end_time'].astype(str), errors='coerce')

    return df


# 각 데이터프레임에 함수 적용
CO = process_dataframe(CO)
CD = process_dataframe(CD)
EO = process_dataframe(EO)
ED = process_dataframe(ED)
WO = process_dataframe(WO)
WD = process_dataframe(WD)
# 데이터프레임 처리 예시
# df = process_dataframe(df) # 데이터프레임에 이 함수 적용


In [ ]:
WD

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts,start_datetime,end_datetime
347,1117051000,1114054000,2023-09-01,08:00:00,08:00:00,1,3,0.0,0.0,0,13990,19,13,2023-09-01 08:00:00,2023-09-01 08:00:00
953,1117053000,1114054000,2023-09-01,12:00:00,13:00:00,0,4,0.0,1.0,1,19979,41,8,2023-09-01 12:00:00,2023-09-01 13:00:00
1594,1114052000,1114054000,2023-09-01,12:00:00,12:00:00,0,2,1.0,1.0,1,17380,9,12,2023-09-01 12:00:00,2023-09-01 12:00:00
1974,1114052000,1114054000,2023-09-01,12:00:00,12:00:00,1,2,4.0,1.0,1,1278,6,34,2023-09-01 12:00:00,2023-09-01 12:00:00
3244,1114055000,1114054000,2023-09-01,12:00:00,12:00:00,0,2,0.0,4.0,4,7137,11,20,2023-09-01 12:00:00,2023-09-01 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3690054,1114052000,1114054000,2023-10-13,17:00:00,17:00:00,1,5,0.0,1.0,3,13285,19,5,2023-10-13 17:00:00,2023-10-13 17:00:00
3690788,1117051000,1114054000,2023-10-13,09:00:00,09:00:00,1,5,0.0,1.0,1,2483,5,5,2023-10-13 09:00:00,2023-10-13 09:00:00
3693728,1114052000,1114054000,2023-10-13,11:00:00,11:00:00,1,3,3.0,1.0,1,1895,7,5,2023-10-13 11:00:00,2023-10-13 11:00:00
3695251,1117053000,1114054000,2023-10-13,17:00:00,18:00:00,1,3,0.0,1.0,1,19429,40,5,2023-10-13 17:00:00,2023-10-13 18:00:00


In [ ]:
def namefy(data):
    #dataset = [CO, CD, EO, ED, WO, WD]
    Data['origin_purpose'] = None
    Data.loc[Data['Trip_purp'] == 4, 'origin_purpose'] = "1_Work" # Work
    Data.loc[Data['origin_purpose'].isin([5,6]), 'origin_purpose'] = "2_Education" # Education
    Data.loc[Data['origin_purpose'].isin([2,7]), 'origin_purpose'] = "3_Business" # Business
    Data.loc[(Data['origin_purpose']>=8)&(Data['origin_purpose']<=11), 'origin_purpose'] = "4_SSL" # Shopping/Social/Lesuire
    Data.loc[Data['origin_purpose']==3, 'origin_purpose'] = "5_Home" # Back home
    Data.loc[Data['origin_purpose'].isin([1,12]), 'origin_purpose'] = "6_Others" # Others

    return Data

In [ ]:
import pandas as pd

# 체류목적 코드와 이동수단 코드를 매핑하는 딕셔너리 생성
purpose_map = {
    0: 'Home',
    1: 'Work',
    2: 'Study',
    3: 'Shopping/Leisure',
    4: 'Other',
    5: 'Travel'
}

modal_map = {
    0: 'Car',
    1: 'Bus',
    2: 'Subway',
    3: 'Walking',
    4: 'Other',
    5: 'Train',
    6: 'Intercity Bus',
    7: 'Airplane'
}

# 함수 정의: 'origin_purpose', 'dest_purpose', 'modal'을 영어로 변환
def map_purpose_modal(df):
    # 'origin_purpose'와 'dest_purpose'에 대해 변환
    df['origin_purpose_eng'] = df['origin_purpose'].map(purpose_map)
    df['dest_purpose_eng'] = df['dest_purpose'].map(purpose_map)
    
    # 'modal'에 대해 변환
    df['modal_eng'] = df['modal'].map(modal_map)
    
    return df

# 데이터셋 리스트
dataset = [CO, CD, EO, ED, WO, WD]

# 각 데이터셋에 변환 함수 적용
for i, df in enumerate(dataset):
    dataset[i] = map_purpose_modal(df)

# 변환된 데이터셋 중 하나 예시 확인 (예: CO)
print(dataset[0][['origin_purpose', 'origin_purpose_eng', 'dest_purpose', 'dest_purpose_eng', 'modal', 'modal_eng']].head())


       origin_purpose origin_purpose_eng  dest_purpose  dest_purpose_eng  \
2228              5.0             Travel             5            Travel   
4999              3.0   Shopping/Leisure             0              Home   
10084             3.0   Shopping/Leisure             3  Shopping/Leisure   
11974             3.0   Shopping/Leisure             3  Shopping/Leisure   
13414             5.0             Travel             5            Travel   

       modal modal_eng  
2228     1.0       Bus  
4999     0.0       Car  
10084    0.0       Car  
11974    0.0       Car  
13414    0.0       Car  


In [ ]:
import pandas as pd

# 각 데이터프레임 CO, CD, EO, ED, WO, WD를 하나로 병합
combined_df = pd.concat([CO, CD ,EO , ED, WO, WD], axis=0, ignore_index=True)

# 병합 후 데이터 확인
print(combined_df.head())


   origin_hdong_cd  dest_hdong_cd       date start_time  end_time  gender  \
0       1114054000     1111061500 2023-09-28   17:00:00  17:00:00       0   
1       1114054000     1117063000 2023-09-28   18:00:00  18:00:00       0   
2       1114054000     1114055000 2023-09-28   13:00:00  13:00:00       1   
3       1114054000     1114057000 2023-09-28   14:00:00  14:00:00       1   
4       1114054000     4128162100 2023-09-28   11:00:00  12:00:00       0   

   age  modal  origin_purpose  dest_purpose  od_dist_avg  od_duration_avg  \
0    0    1.0             5.0             5        11789               13   
1    3    0.0             3.0             0        16476               22   
2    3    0.0             3.0             3         9990               23   
3    2    0.0             3.0             3        15297               23   
4    0    0.0             5.0             5        21140               68   

   od_cnts      start_datetime        end_datetime origin_purpose_eng  \
0

In [ ]:
combined_df.sort_values(by='start_datetime', inplace=True)

In [ ]:
combined_df.isnull().sum()

origin_hdong_cd        0
dest_hdong_cd          0
date                   0
start_time             0
end_time               0
gender                 0
age                    0
modal                  0
origin_purpose        40
dest_purpose           0
od_dist_avg            0
od_duration_avg        0
od_cnts                0
start_datetime         0
end_datetime           0
origin_purpose_eng    40
dest_purpose_eng       0
modal_eng              0
dtype: int64

In [ ]:
combined_df.dropna(subset=['origin_hdong_cd', 'dest_hdong_cd'], inplace=True)

In [ ]:
combined_df = combined_df.drop_duplicates()

In [ ]:
combined_df 

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts,start_datetime,end_datetime,origin_purpose_eng,dest_purpose_eng,modal_eng
143593,1117055500,1114054000,2023-09-01,08:00:00,08:00:00,1,3,0.0,0.0,1,29769,27,5,2023-09-01 08:00:00,2023-09-01 08:00:00,Home,Work,Car
142546,4131051000,1114054000,2023-09-01,08:00:00,10:00:00,1,3,2.0,0.0,1,56271,97,5,2023-09-01 08:00:00,2023-09-01 10:00:00,Home,Work,Subway
144074,1132068000,1114054000,2023-09-01,08:00:00,09:00:00,0,5,2.0,0.0,1,79440,78,6,2023-09-01 08:00:00,2023-09-01 09:00:00,Home,Work,Subway
144640,1174065000,1114054000,2023-09-01,08:00:00,09:00:00,1,5,2.0,0.0,1,28265,50,7,2023-09-01 08:00:00,2023-09-01 09:00:00,Home,Work,Subway
46711,1114054000,1114052000,2023-09-01,08:00:00,08:00:00,0,4,0.0,4.0,4,3171,6,5,2023-09-01 08:00:00,2023-09-01 08:00:00,Other,Other,Car
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29385,1114054000,1114052000,2023-10-15,23:00:00,23:00:00,1,1,0.0,0.0,0,2604,11,19,2023-10-15 23:00:00,2023-10-15 23:00:00,Home,Home,Car
29544,1114054000,1114052000,2023-10-15,23:00:00,23:00:00,1,1,0.0,4.0,4,1749,5,10,2023-10-15 23:00:00,2023-10-15 23:00:00,Other,Other,Car
29582,1114054000,1111067000,2023-10-15,23:00:00,23:00:00,1,3,0.0,5.0,5,7381,21,6,2023-10-15 23:00:00,2023-10-15 23:00:00,Travel,Travel,Car
28793,1114054000,1117051000,2023-10-15,23:00:00,23:00:00,0,5,0.0,4.0,4,15128,29,6,2023-10-15 23:00:00,2023-10-15 23:00:00,Other,Other,Car


In [ ]:
combined_df.to_csv('combined_data.csv', index=False)
WD.to_csv('WD.csv', index=False)
WO.to_csv('WO.csv', index=False)
ED.to_csv('ED.csv', index=False)
EO.to_csv('EO.csv', index=False)
CD.to_csv('CD.csv', index=False)
CO.to_csv('CO.csv', index=False)

In [ ]:
import glob

# 'rawdata' 디렉토리에서 'od'로 시작하는 모든 CSV 파일 경로 찾기
file_paths = glob.glob('rawdata/od*.csv')

# 합쳐질 CSV 파일 경로 지정
output_file = 'rawdata/combined_od_data.csv'

# 첫 번째 파일의 헤더 포함하여 기록
with open(output_file, 'w') as outfile:
    for i, file_path in enumerate(file_paths):
        with open(file_path, 'r') as infile:
            if i == 0:
                # 첫 번째 파일의 헤더를 포함하여 기록
                outfile.write(infile.read())
            else:
                # 나머지 파일들은 헤더를 제외하고 기록
                next(infile)  # 첫 번째 줄(헤더) 건너뜀
                outfile.write(infile.read())

print(f"모든 파일이 {output_file}에 성공적으로 병합되었습니다.")


모든 파일이 rawdata/combined_od_data.csv에 성공적으로 병합되었습니다.


In [ ]:
data = pd.read_csv('rawdata/combined_od_data.csv')

In [ ]:
# date 열을 문자열에서 datetime 형식으로 변환
NData['date'] = pd.to_datetime(NData['date'], format='%Y%m%d')

# start_time을 문자열로 변환하고 시와 분을 나타내는 시간으로 변환
NData['start_time'] = NData['start_time'].astype(str) + ':00'

# date와 start_time을 합쳐서 하나의 datetime으로 변환
NData['datetime'] = pd.to_datetime(NData['date'].dt.strftime('%Y-%m-%d') + ' ' + NData['start_time'], format='%Y-%m-%d %H:%M')

# 필요에 따라 기존 열 제거 가능
NData = NData.drop(columns=['date', 'start_time'])




In [ ]:
NData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3846 entries, 1025581 to 1227504
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   origin_hdong_cd  3846 non-null   int64         
 1   dest_hdong_cd    3846 non-null   int64         
 2   end_time         3846 non-null   int32         
 3   gender           3846 non-null   int64         
 4   age              3846 non-null   int64         
 5   modal            3846 non-null   float64       
 6   origin_purpose   3846 non-null   float64       
 7   dest_purpose     3846 non-null   int64         
 8   od_dist_avg      3846 non-null   int64         
 9   od_duration_avg  3846 non-null   int64         
 10  od_cnts          3846 non-null   int64         
 11  datetime         3846 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int32(1), int64(8)
memory usage: 375.6 KB


In [ ]:
dongcode = pd.read_csv('rawdata/KIKmix_20230701.csv')

In [4]:
combined_data

NameError: name 'combined_data' is not defined

In [ ]:

# dongcode에서 행정동코드가 고유한지 확인하고 중복 제거
dongcode_unique = dongcode.drop_duplicates(subset=['행정동코드'])

# origin_hdong_cd와 dest_hdong_cd를 기준으로 병합
combined_df1 = combined_df.merge(dongcode_unique[['행정동코드', '시도명', '시군구명', '읍면동명']], left_on='origin_hdong_cd', right_on='행정동코드', how='left')
combined_df1 = combined_df1.rename(columns={'시도명': 'origin_시도명', '시군구명': 'origin_시군구명', '읍면동명': 'origin_읍면동명'})
combined_df1 = combined_df1.drop(columns=['행정동코드'])

combined_df1 = combined_df1.merge(dongcode_unique[['행정동코드', '시도명', '시군구명', '읍면동명']], left_on='dest_hdong_cd', right_on='행정동코드', how='left')
combined_df1 = combined_df1.rename(columns={'시도명': 'dest_시도명', '시군구명': 'dest_시군구명', '읍면동명': 'dest_읍면동명'})
combined_df1 = combined_df1.drop(columns=['행정동코드'])

# 결과 확인
print(combined_df1)
print(f"병합 후 행의 수: {len(combined_df1)}")

        origin_hdong_cd  dest_hdong_cd       date start_time  end_time  \
0            1117055500     1114054000 2023-09-01   08:00:00  08:00:00   
1            4131051000     1114054000 2023-09-01   08:00:00  10:00:00   
2            1132068000     1114054000 2023-09-01   08:00:00  09:00:00   
3            1174065000     1114054000 2023-09-01   08:00:00  09:00:00   
4            1114054000     1114052000 2023-09-01   08:00:00  08:00:00   
...                 ...            ...        ...        ...       ...   
214103       1114054000     1114052000 2023-10-15   23:00:00  23:00:00   
214104       1114054000     1114052000 2023-10-15   23:00:00  23:00:00   
214105       1114054000     1111067000 2023-10-15   23:00:00  23:00:00   
214106       1114054000     1117051000 2023-10-15   23:00:00  23:00:00   
214107       1114054000     1117051000 2023-10-15   23:00:00  23:00:00   

        gender  age  modal  origin_purpose  dest_purpose  ...  \
0            1    3    0.0             0.0    

In [ ]:
combined_df1

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,...,end_datetime,origin_purpose_eng,dest_purpose_eng,modal_eng,origin_시도명,origin_시군구명,origin_읍면동명,dest_시도명,dest_시군구명,dest_읍면동명
0,1117055500,1114054000,2023-09-01,08:00:00,08:00:00,1,3,0.0,0.0,1,...,2023-09-01 08:00:00,Home,Work,Car,서울특별시,용산구,청파동,서울특별시,중구,회현동
1,4131051000,1114054000,2023-09-01,08:00:00,10:00:00,1,3,2.0,0.0,1,...,2023-09-01 10:00:00,Home,Work,Subway,경기도,구리시,갈매동,서울특별시,중구,회현동
2,1132068000,1114054000,2023-09-01,08:00:00,09:00:00,0,5,2.0,0.0,1,...,2023-09-01 09:00:00,Home,Work,Subway,서울특별시,도봉구,쌍문제3동,서울특별시,중구,회현동
3,1174065000,1114054000,2023-09-01,08:00:00,09:00:00,1,5,2.0,0.0,1,...,2023-09-01 09:00:00,Home,Work,Subway,서울특별시,강동구,성내제2동,서울특별시,중구,회현동
4,1114054000,1114052000,2023-09-01,08:00:00,08:00:00,0,4,0.0,4.0,4,...,2023-09-01 08:00:00,Other,Other,Car,서울특별시,중구,회현동,서울특별시,중구,소공동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214103,1114054000,1114052000,2023-10-15,23:00:00,23:00:00,1,1,0.0,0.0,0,...,2023-10-15 23:00:00,Home,Home,Car,서울특별시,중구,회현동,서울특별시,중구,소공동
214104,1114054000,1114052000,2023-10-15,23:00:00,23:00:00,1,1,0.0,4.0,4,...,2023-10-15 23:00:00,Other,Other,Car,서울특별시,중구,회현동,서울특별시,중구,소공동
214105,1114054000,1111067000,2023-10-15,23:00:00,23:00:00,1,3,0.0,5.0,5,...,2023-10-15 23:00:00,Travel,Travel,Car,서울특별시,중구,회현동,서울특별시,종로구,창신제1동
214106,1114054000,1117051000,2023-10-15,23:00:00,23:00:00,0,5,0.0,4.0,4,...,2023-10-15 23:00:00,Other,Other,Car,서울특별시,중구,회현동,서울특별시,용산구,후암동


In [168]:
combined_df1.to_csv('combined_df.csv', index=False)

In [164]:
# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 필요한 패키지와 라이브러리를 가져옴
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False
print ('버전: ', mpl.__version__)
print ('설치 위치: ', mpl.__file__)
print ('설정 위치: ', mpl.get_configdir())
print ('캐시 위치: ', mpl.get_cachedir())
print ('설정파일 위치: ', mpl.matplotlib_fname())
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')

# ttf 폰트 전체갯수
print(len(font_list)) 
[(f.name, f.fname) for f in fm.fontManager.ttflist if 'Gothic' in f.name]

버전:  3.9.2
설치 위치:  c:\Users\ansck\anaconda3\envs\bitcoin\Lib\site-packages\matplotlib\__init__.py
설정 위치:  C:\Users\ansck\.matplotlib
캐시 위치:  C:\Users\ansck\.matplotlib
설정파일 위치:  c:\Users\ansck\anaconda3\envs\bitcoin\Lib\site-packages\matplotlib\mpl-data\matplotlibrc
411


[('Copperplate Gothic Light', 'C:\\Windows\\Fonts\\COPRGTL.TTF'),
 ('Yu Gothic', 'C:\\Windows\\Fonts\\YuGothL.ttc'),
 ('HYGothic-Extra', 'C:\\Windows\\Fonts\\H2GTRE.TTF'),
 ('HYGothic-Medium', 'C:\\Windows\\Fonts\\H2GTRM.TTF'),
 ('AppleSDGothicNeoH00',
  'C:\\Users\\ansck\\AppData\\Local\\Microsoft\\Windows\\Fonts\\AppleSDGothicNeoH.ttf'),
 ('Franklin Gothic Medium', 'C:\\Windows\\Fonts\\framd.ttf'),
 ('Showcard Gothic', 'C:\\Windows\\Fonts\\SHOWG.TTF'),
 ('AppleSDGothicNeoEB00',
  'C:\\Users\\ansck\\AppData\\Local\\Microsoft\\Windows\\Fonts\\AppleSDGothicNeoEB.ttf'),
 ('Franklin Gothic Medium Cond', 'C:\\Windows\\Fonts\\FRAMDCN.TTF'),
 ('Franklin Gothic Demi', 'C:\\Windows\\Fonts\\FRADM.TTF'),
 ('Century Gothic', 'C:\\Windows\\Fonts\\GOTHICB.TTF'),
 ('Franklin Gothic Book', 'C:\\Windows\\Fonts\\FRABKIT.TTF'),
 ('Franklin Gothic Heavy', 'C:\\Windows\\Fonts\\FRAHVIT.TTF'),
 ('AppleSDGothicNeoM00',
  'C:\\Users\\ansck\\AppData\\Local\\Microsoft\\Windows\\Fonts\\AppleSDGothicNeoM.ttf'),
 